A simple analysis of this truss is performed by considering the force balance equations.  This may be expressed as a matrix which relates the unknown internal forces on all beams to the known externally applied forces (i.e. the truck).  The reaction forces pushing on the bridge are regarded as internal forces since they are unknown.  We write this relation as

$Af_i=−f_e$

Where $f_i$ is a vector (list) of the internal forces on each beam, and fe is the vector of known external forces. The matrix A holds the information detailing how the forces interact, i.e. the structure of the bridge.  The goal is to find the internal forces $f_i$.

![truss.png](truss.png)

## 1. Complete the above analysis and write down the entire matrix equation describing the forces at each joint.

### Left positive, right negative
$$a_x \rightarrow -f_1cos(45)-f_2+r_{ax} = 0$$

$$a_y \rightarrow -f_1sin(45)+r_{ay} = 0$$

$$b_x \rightarrow f_1cos(45) + f_4 = 0$$

$$b_y \rightarrow f_1sin(45) + f_3 = 0 $$

$$c_x \rightarrow f_2 - f_5cos(45) - f_6 = 0$$

$$c_y \rightarrow -f_3 - f_5sin(45) = -T_c$$

$$d_x \rightarrow f_4 + f_5cos(45) - f_8cos(45) = 0$$

$$d_y \rightarrow f_5sin(45) + f_7 + f_8sin(45) $$

$$e_x \rightarrow f_6 - f_9 = 0$$

$$e_y \rightarrow -f_7 = -T_e$$

$$f_x \rightarrow f_8 + f_9 = 0$$

$$f_y \rightarrow -f_8sin(45) + r_{fy} = 0$$

In [51]:
import math
import pandas as pd
from scipy import sparse

COLS = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'Rax', 'Ray', 'Rfy']

sin45 = math.sin(math.pi / 4)
cos45 = math.cos(math.pi / 4)

eqs = {
    'ax': {'f1': -cos45, 'f2': -1, 'Rax': 1},
    'ay': {'f1': -sin45, 'Ray': 1},
    'bx': {'f1': cos45, 'f4': 1},
    'by': {'f1': sin45, 'f3': 1},
    'cx': {'f2': 1, 'f5': -cos45, 'f6': -1},
    'cy': {'f3': -1, 'f5': -sin45},
    'dx': {'f4': 1, 'f5': cos45, 'f8': -cos45},
    'dy': {'f5': sin45, 'f7': 1, 'f8': sin45},
    'ex': {'f6': 1, 'f9': -1},
    'ey': {'f7': -1},
    'fx': {'f8': 1, 'f9': 1},
    'fy': {'f8': -sin45, 'Rfy': 1}
}

rows = []
cols = []
data = []

for i, key in enumerate(eqs.keys()):
    for x, row in enumerate(eqs[key].keys()):
        rows.append(i)
        cols.append(COLS.index(row))
        data.append(eqs[key][row])


mat = sparse.coo_matrix((data, (rows, cols)), shape=(len(eqs), len(COLS)))
df = pd.DataFrame(mat.toarray(), columns=COLS)
df['row'] = eqs.keys()
display(df.set_index('row'))

import numpy as np
ans = np.zeros(len(eqs))
from scipy.sparse.linalg import spsolve

truck_weight = 50
Tc = Te = truck_weight / 2
ans[5] = -Tc
ans[9] = -Te
pd.DataFrame(list(zip(COLS, spsolve(mat, ans)))).set_index(0)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,Rax,Ray,Rfy
row,,,,,,,,,,,,
ax,-0.707107,-1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
ay,-0.707107,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.0,0.0
bx,0.707107,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
by,0.707107,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cx,0.000000,1.0,0.0,0.0,-0.707107,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0
cy,0.000000,0.0,-1.0,0.0,-0.707107,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
dx,0.000000,0.0,0.0,1.0,0.707107,0.0,0.0,-0.707107,0.0,0.0,0.0,0.0
dy,0.000000,0.0,0.0,0.0,0.707107,0.0,1.0,0.707107,0.0,0.0,0.0,0.0
ex,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,-1.0,0.0,0.0,0.0


/Users/anirudhkamath/opt/anaconda3/envs/MATH7203/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


,1
0,
f1,-106.066017
f2,-85.355339
f3,75.000000
f4,75.000000
f5,-70.710678
f6,-35.355339
f7,25.000000
f8,35.355339
f9,-35.355339
